In [41]:
import netCDF4
import pandas
import numpy
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [125]:
# read data from the input (POES) file
testFiles = ['/home/bharat/Desktop/poesTest/poes_n18_20150617_proc.nc', \
                '/home/bharat/Desktop/poesTest/poes_m01_20150617_proc.nc', \
                '/home/bharat/Desktop/poesTest/poes_n19_20150617_proc.nc', \
                '/home/bharat/Desktop/poesTest/poes_m02_20150617_proc.nc', 
                '/home/bharat/Desktop/poesTest/poes_n15_20150617_proc.nc']
inpFil = testFiles[0]
poesRawData = netCDF4.Dataset(inpFil)
poesRawData2 = netCDF4.Dataset(testFiles[1])
# for i in poesRawData.variables.iterkeys():
#     print i
a = numpy.round( poesRawData.variables['aacgm_lon_foot'][:], decimals=1 )
print poesRawData.variables['ted_ele_energy_tel0'][:150]

[ 0  2  8  8  8 15  0  8  8  0  0  0  8  0 13  8  8  8  0  8  0  0  8  0  0
 14  8 11  0  0  8  2  0  8  8  8  0 11  0  0  8  8  8  0  7  8 14  8  8  0
  0  8  0 10  8  0  8 12  0  0  0  0  8  5  0  8  8  0  0  0  0  0  8  0 12
  0  8  8  0  0  0 11  8  0 13  8  0  2  0  1  1  0  0  8  0  9  0 15  0  5
  0  0  8  0  8 12  0 14  6  8  0  8  5  8  0  8  8  8  8  0  0  8  0  0  8
  8  8  0  0  8  3 14  8  0  0 11  0 12 15  0 15  8  8  0  0  0  8 12  0  0]


In [43]:
poesDF = pandas.DataFrame( poesRawData.variables['time'][:], columns=[ "timestamp" ] )
poesDF['date'] = pandas.to_datetime(poesDF['timestamp'], unit='ms')
poesDF["alt"] = poesRawData.variables['alt'][:]
poesDF["aacgm_lat_foot"] = poesRawData.variables['aacgm_lat_foot'][:]
poesDF["aacgm_lon_foot"] = poesRawData.variables['aacgm_lon_foot'][:]
poesDF["MLT"] = poesRawData.variables['MLT'][:]
# Add up the fluxes
poesDF["ted_ele_total_flux"] = poesRawData.variables['ted_ele_tel0_flux_4'][:] +\
        poesRawData.variables['ted_ele_tel0_flux_8'][:] + \
        poesRawData.variables['ted_ele_tel0_flux_11'][:] + \
        poesRawData.variables['ted_ele_tel0_flux_14'][:] + \
        poesRawData.variables['ted_ele_tel30_flux_4'][:] +\
        poesRawData.variables['ted_ele_tel30_flux_8'][:] + \
        poesRawData.variables['ted_ele_tel30_flux_11'][:] + \
        poesRawData.variables['ted_ele_tel30_flux_14'][:]
poesDF["ted_pro_total_flux"] = poesRawData.variables['ted_pro_tel0_flux_4'][:] +\
        poesRawData.variables['ted_pro_tel0_flux_8'][:] + \
        poesRawData.variables['ted_pro_tel0_flux_11'][:] + \
        poesRawData.variables['ted_pro_tel0_flux_14'][:] + \
        poesRawData.variables['ted_pro_tel30_flux_4'][:] +\
        poesRawData.variables['ted_pro_tel30_flux_8'][:] + \
        poesRawData.variables['ted_pro_tel30_flux_11'][:] + \
        poesRawData.variables['ted_pro_tel30_flux_14'][:]
# filter out unwanted data (0's and -1's in fluxs)
poesDF = poesDF[ (poesDF["ted_pro_total_flux"] > 0)\
                | (poesDF["ted_ele_total_flux"] > 0) ].reset_index()
poesDF['log_ele_flux'] = [0. if x <= 0. else numpy.log10(x) for x in poesDF['ted_ele_total_flux']]
poesDF['log_pro_flux'] = [0. if x <= 0. else numpy.log10(x) for x in poesDF['ted_pro_total_flux']]
poesDF["sat"] = "n18"
# seperate out electron and proton flux and discard all zeros
poesEleFluxDF = poesDF[poesDF["log_ele_flux"] > 0.][ ["timestamp", "date", "aacgm_lat_foot",\
                         "aacgm_lon_foot", "MLT", "log_ele_flux", "sat"] ].reset_index(drop=True)
poesProFluxDF = poesDF[poesDF["log_pro_flux"] > 0.][ ["timestamp", "date", "aacgm_lat_foot",\
                         "aacgm_lon_foot", "MLT", "log_pro_flux", "sat"] ].reset_index(drop=True)

# poesProFluxDF.head()

In [44]:
poesDF2 = pandas.DataFrame( poesRawData2.variables['time'][:], columns=[ "timestamp" ] )
poesDF2['date'] = pandas.to_datetime(poesDF2['timestamp'], unit='ms')
poesDF2["alt"] = poesRawData2.variables['alt'][:]
poesDF2["aacgm_lat_foot"] = poesRawData2.variables['aacgm_lat_foot'][:]
poesDF2["aacgm_lon_foot"] = poesRawData2.variables['aacgm_lon_foot'][:]
poesDF2["MLT"] = poesRawData2.variables['MLT'][:]
# Add up the fluxes
poesDF2["ted_ele_total_flux"] = poesRawData2.variables['ted_ele_tel0_flux_4'][:] +\
        poesRawData2.variables['ted_ele_tel0_flux_8'][:] + \
        poesRawData2.variables['ted_ele_tel0_flux_11'][:] + \
        poesRawData2.variables['ted_ele_tel0_flux_14'][:] + \
        poesRawData2.variables['ted_ele_tel30_flux_4'][:] +\
        poesRawData2.variables['ted_ele_tel30_flux_8'][:] + \
        poesRawData2.variables['ted_ele_tel30_flux_11'][:] + \
        poesRawData2.variables['ted_ele_tel30_flux_14'][:]
poesDF2["ted_pro_total_flux"] = poesRawData2.variables['ted_pro_tel0_flux_4'][:] +\
        poesRawData2.variables['ted_pro_tel0_flux_8'][:] + \
        poesRawData2.variables['ted_pro_tel0_flux_11'][:] + \
        poesRawData2.variables['ted_pro_tel0_flux_14'][:] + \
        poesRawData2.variables['ted_pro_tel30_flux_4'][:] +\
        poesRawData2.variables['ted_pro_tel30_flux_8'][:] + \
        poesRawData2.variables['ted_pro_tel30_flux_11'][:] + \
        poesRawData2.variables['ted_pro_tel30_flux_14'][:]
# filter out unwanted data (0's and -1's in fluxs)
poesDF2 = poesDF2[ (poesDF2["ted_pro_total_flux"] > 0)\
                | (poesDF2["ted_ele_total_flux"] > 0) ].reset_index(drop=True)
poesDF2['log_ele_flux'] = [0. if x <= 0. else numpy.log10(x) for x in poesDF2['ted_ele_total_flux']]
poesDF2['log_pro_flux'] = [0. if x <= 0. else numpy.log10(x) for x in poesDF2['ted_pro_total_flux']]
poesDF2["sat"] = "m01"
# seperate out electron and proton flux and discard all zeros
poesEleFluxDF2 = poesDF2[poesDF2["log_ele_flux"] > 0.][ ["timestamp", "date", "aacgm_lat_foot",\
                         "aacgm_lon_foot", "MLT", "log_ele_flux", "sat"] ].reset_index()
poesProFluxDF2 = poesDF2[poesDF2["log_pro_flux"] > 0.][ ["timestamp", "date", "aacgm_lat_foot",\
                         "aacgm_lon_foot", "MLT", "log_pro_flux", "sat"] ].reset_index(drop=True)

# poesProFluxDF2.head()

In [70]:
poesAllEleData = pandas.DataFrame( columns =  ["timestamp", "date", "aacgm_lat_foot",\
                         "aacgm_lon_foot", "MLT", "log_ele_flux", "sat"] )
poesAllProData = pandas.DataFrame( columns =  ["timestamp", "date", "aacgm_lat_foot",\
                         "aacgm_lon_foot", "MLT", "log_pro_flux", "sat"] )

poesAllEleData = poesAllEleData.append( poesEleFluxDF )#pandas.concat( [ poesAllEleData, poesEleFluxDF ] )
poesAllEleData = poesAllEleData.append( poesEleFluxDF2 )#pandas.concat( [ poesAllEleData, poesEleFluxDF2 ] )
poesEleFluxDF[ poesEleFluxDF["sat"] == "n18" ].head()
# poesProFluxDF2.head()
# poesProFluxAll = pandas.concat( [ poesProFluxDF, poesProFluxDF2 ] )
# poesProFluxAll.combine
# poesProFluxDF.size

,timestamp,date,aacgm_lat_foot,aacgm_lon_foot,MLT,log_ele_flux,sat
0,1434499228215,2015-06-17 00:00:28.215,53.950001,309.739990,16.209999,1.370698,n18
1,1434499278214,2015-06-17 00:01:18.214,56.369999,307.679993,16.080000,1.072618,n18
2,1434499348214,2015-06-17 00:02:28.214,59.720001,304.369995,15.880000,1.935356,n18
3,1434499374214,2015-06-17 00:02:54.214,60.939999,303.000000,15.800000,1.072618,n18
4,1434499422214,2015-06-17 00:03:42.214,63.160000,300.179993,15.630000,2.449787,n18


In [85]:
eleInpFile = "/home/bharat/Desktop/poesTest/eleflux.csv"
proInpFile = "/home/bharat/Desktop/poesTest/proflux.csv"
eleDF = pandas.read_csv(eleInpFile, sep=' ')
proDF = pandas.read_csv(proInpFile, sep=' ')
eleDF["date"] = pandas.to_datetime( eleDF["date"] )
proDF["date"] = pandas.to_datetime( proDF["date"] )

In [92]:
# select data for a datetime range
selDate = datetime.datetime( 2015,6,17,9 ) 
timeInterval = datetime.timedelta( minutes=30 )
eleDF = eleDF[ (eleDF["date"] > selDate-timeInterval)\
                & (eleDF["date"] < selDate+timeInterval) ]
# poesProFluxDF.plot( x="aacgm_lat_foot", y="log_pro_flux", kind="scatter" )
eleDF[eleDF["sat"] == "n15"].head(100)

,timestamp,date,aacgm_lat_foot,aacgm_lon_foot,MLT,log_ele_flux,sat,dateStr,time
61530,1.434530e+12,2015-06-17 08:30:04.189,-76.339996,254.630005,20.400000,2.343999,n15,20150617,830
61531,1.434530e+12,2015-06-17 08:30:06.189,-76.269997,254.250000,20.370001,2.951289,n15,20150617,830
61532,1.434530e+12,2015-06-17 08:30:12.190,-76.050003,253.100006,20.299999,2.648945,n15,20150617,830
61533,1.434530e+12,2015-06-17 08:30:14.190,-75.980003,252.720001,20.270000,2.951289,n15,20150617,830
61534,1.434530e+12,2015-06-17 08:30:20.189,-75.750000,251.589996,20.200001,3.049218,n15,20150617,830
61535,1.434530e+12,2015-06-17 08:30:22.189,-75.680000,251.210007,20.170000,2.470263,n15,20150617,830
61536,1.434530e+12,2015-06-17 08:30:28.189,-75.449997,250.119995,20.100000,2.433177,n15,20150617,830
61537,1.434530e+12,2015-06-17 08:30:30.189,-75.370003,249.800003,20.080000,3.184175,n15,20150617,830
61538,1.434530e+12,2015-06-17 08:30:36.190,-75.129997,248.839996,20.020000,2.648945,n15,20150617,830
61539,1.434530e+12,2015-06-17 08:30:38.190,-75.050003,248.520004,20.000000,3.253289,n15,20150617,830
